In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB 
from nltk.stem.snowball import FrenchStemmer
from xgboost import XGBClassifier

In [2]:
Corpus= pd.read_csv('./nlp/tweets.csv', sep='^([^,]+),', engine='python', error_bad_lines=False, encoding='utf-8', index_col=[0])

In [3]:
Corpus.columns=['polarity','text']

In [4]:
Corpus = Corpus.sample(n=1000)
Corpus = Corpus.reset_index(drop=True)

In [5]:
Corpus['text']=Corpus['text'].astype(str)
Corpus['text'] = [entry.lower() for entry in Corpus['text']]
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['text']):
    Final_words = []
    stemmer = FrenchStemmer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('french') and word.isalpha():
            word_Final = stemmer.stem(word)
            Final_words.append(word_Final)
    Corpus.loc[index,'text_final'] = str(Final_words)

In [6]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['polarity'],test_size=0.3)

In [7]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [8]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [9]:
rfc = RandomForestClassifier()
rfc.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_rfc = rfc.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("Random Forest Accuracy Score -> ",accuracy_score(predictions_rfc, Test_Y)*100)
#score 60% pour sample 1000 

Random Forest Accuracy Score ->  59.66666666666667


C:\Users\aline\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
